In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [26]:
import torch
from datasets import load_dataset
from collections import Counter
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

In [4]:
# Set the seed
seed = 42
torch.manual_seed(seed)
# Probably, this below must be changed if you work with a M1/M2/M3 Mac
torch.cuda.manual_seed(seed) # for CUDA
torch.backends.cudnn.deterministic = True # for CUDNN
torch.backends.benchmark = False # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.

## Data

### Question 1

In [5]:
# Question 1
ds = load_dataset("heegyu/news-category-dataset")
print(ds['train'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/101 [00:00<?, ?B/s]

data.json:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/209527 [00:00<?, ? examples/s]

Dataset({
    features: ['link', 'headline', 'category', 'short_description', 'authors', 'date'],
    num_rows: 209527
})


### Question 2

In [6]:
# Question 2
# Filter for "POLITICS" category and store each headline as a string in ds_train
ds_train = [news['headline'] for news in ds['train'] if news['category'] == 'POLITICS']

assert len(ds_train) == 35602

print("First headline (before processing):", ds_train[0])

First headline (before processing): Biden Says U.S. Forces Would Defend Taiwan If China Invaded


### Question 3

In [7]:
# Convert each headline to lowercase
ds_train = [headline.lower() for headline in ds_train]

# Check the result
print(ds_train[0])

# Split each headline in words
# maybe I could use a better tokenizer (ex. remove all punctation)
ds_train = [headline.split(" ") for headline in ds_train]

# Check the result
print(ds_train[0])

biden says u.s. forces would defend taiwan if china invaded
['biden', 'says', 'u.s.', 'forces', 'would', 'defend', 'taiwan', 'if', 'china', 'invaded']


In [8]:
# Add <EOS> at the end of every headline
for headline in ds_train:
    headline.append('<EOS>')

# Check the result
print(ds_train[0])

['biden', 'says', 'u.s.', 'forces', 'would', 'defend', 'taiwan', 'if', 'china', 'invaded', '<EOS>']


In [9]:
print(ds_train[:5])

[['biden', 'says', 'u.s.', 'forces', 'would', 'defend', 'taiwan', 'if', 'china', 'invaded', '<EOS>'], ['‘beautiful', 'and', 'sad', 'at', 'the', 'same', 'time’:', 'ukrainian', 'cultural', 'festival', 'takes', 'on', 'a', 'deeper', 'meaning', 'this', 'year', '<EOS>'], ['biden', 'says', "queen's", 'death', 'left', "'giant", "hole'", 'for', 'royal', 'family', '<EOS>'], ['bill', 'to', 'help', 'afghans', 'who', 'escaped', 'taliban', 'faces', 'long', 'odds', 'in', 'the', 'senate', '<EOS>'], ['mark', 'meadows', 'complies', 'with', 'justice', 'dept.', 'subpoena:', 'report', '<EOS>']]


### Question 4

In [10]:
# Flatten ds_train and extract all words (including <EOS> and PAD tokens)
all_words = [word for headline in ds_train for word in headline]

# Count word frequencies
word_counts = Counter(all_words)

# Get the 5 most common words
most_common_words = word_counts.most_common(5)

# Print the 5 most common words
print("5 most common words:", most_common_words)

5 most common words: [('<EOS>', 35602), ('to', 10701), ('the', 9618), ('trump', 6895), ('of', 5536)]


In [11]:
# Flatten ds_train and extract unique words
unique_words = set(word for headline in ds_train for word in headline)

# Create vocabulary with <EOS> at the beginning and PAD at the end and remove evenutally alredy presents special tokens
unique_words = {word for word in unique_words if word and word not in ["<EOS>", "PAD"]}

# Sorting of unique_words
word_vocab = ["<EOS>"] + sorted(list(unique_words)) + ["PAD"]

# Total number of unique words (excluding <EOS> and PAD)
total_words = len(word_vocab) - 2

# Print the total number of words in the vocabulary
print("Total number of words in vocabulary (excluding <EOS> and PAD):", total_words)

# Remove words that are used less than a threshold (5 times):
threshold = 5
filtered_words = {word for word, count in word_counts.items() if count >= threshold}
filtered_word_vocab = ["<EOS>"] + sorted(list(filtered_words)) + ["PAD"]

# Number of unique words after filtering (excluding <EOS> and PAD)
total_words = len(filtered_word_vocab) - 2

# Print the total number of words in the vocabulary
print("Total number of words in vocabulary after filtering (excluding <EOS> and PAD):", total_words)

Total number of words in vocabulary (excluding <EOS> and PAD): 33230
Total number of words in vocabulary after filtering (excluding <EOS> and PAD): 7034


In [12]:
# Dictionary representing a mapping from words of our word_vocab to integer values
word_to_int = {word: i for i, word in enumerate(word_vocab)}

assert word_to_int['<EOS>'] == 0 and word_to_int['PAD'] == len(word_vocab) - 1
#print(f"<EOS> index: {word_to_int['<EOS>']}")
#print(f"PAD index: {word_to_int['PAD']}")

# print("Sample mapping:", list(word_to_int.items())[:10])  # Print first 10 mappings

# Dictionary representing the inverse of `word_to_int`, i.e. a mapping from integer (keys) to characters (values).
int_to_word = {word:i for i, word in word_to_int.items()}

assert int_to_word[0] == '<EOS>' and int_to_word[len(word_vocab)-1] == 'PAD'
#print(f"Word at first index (0): {int_to_word[0]}")
#print(f"Word at last index ({len(word_vocab)-1}): {int_to_word[len(word_vocab)-1]}")

# print("Sample mapping:", list(int_to_word.items())[:10])  # Print first 10 mappings

### Question 5

In [13]:
# Create a dataset class
# - input:
#       list of tokenized sequences
#       word_to_int
# - Each item: a tuple having
#        the indexes of all the words of the sentence except the last one;
#        all the elements of that sentence except the first one

class Dataset:
    def __init__(self, sequences, word_to_int):
        self.sequences = sequences
        self.word_to_int = word_to_int

        # Convert each sequence (list of words) to indexes using map
        self.indexed_sequences = [
            [self.word_to_int[word] for word in sequence if word in self.word_to_int]
            for sequence in self.sequences
        ] # the problem is that if in the sequence there is a word (ex '') without mapping, skip it

    def __getitem__(self, idx):
        # Get the indexed sequence at the given index
        indexed_seq = self.indexed_sequences[idx]

        # Create x (all indexes except the last one) and y (all indexes except the first one)
        x = indexed_seq[:-1]
        y = indexed_seq[1:]

        return torch.tensor(x), torch.tensor(y)

    def __len__(self):
        # Return the total number of sequences
        return len(self.indexed_sequences)

### Question 6

In [14]:
def collate_fn(batch, pad_value):
  # Separate data (x) and target (y) pairs from the batch
  data, targets = zip(*batch)

  padded_data = nn.utils.rnn.pad_sequence(data, batch_first=True, padding_value=pad_value)
  padded_targets = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=pad_value)

  return padded_data, padded_targets

In [15]:
batch_size = 8

dataset = Dataset(ds_train, word_to_int)

if batch_size == 1:
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
else:
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=lambda batch: collate_fn(batch, word_to_int["PAD"]))

  # By default, DataLoader expects a function like collate_fn(batch) that takes only one argument—the batch itself.
  # However, in this case, collate_fn requires an additional argument (pad_value).
  # The lambda function allows to rewrite collate_fn(batch, pad_value) into a version compatible with DataLoader

## Model definition

In [16]:
class LSTMModel(nn.Module):
    def __init__(self, map, hidden_size=1024, emb_dim=150, n_layers=1):
        super(LSTMModel, self).__init__()

        self.vocab_size  = len(map)
        self.hidden_size = hidden_size
        self.emb_dim     = emb_dim
        self.n_layers    = n_layers

        # Embedding layer
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim=self.emb_dim,
            padding_idx=map["PAD"]
        )

        # LSTM layer with potential stacking
        self.lstm = nn.LSTM(
            input_size=self.emb_dim,
            hidden_size=self.hidden_size,
            num_layers=self.n_layers,
            batch_first=True
        )

        # Fully connected layer to project LSTM outputs to vocabulary size
        self.fc = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.vocab_size
        )

    def forward(self, x, prev_state):
        # Embedding lookup for input tokens
        embed = self.embedding(x)

        # Pass embeddings through the LSTM
        yhat, state = self.lstm(embed, prev_state)  # yhat: (batch, seq_length, hidden_size)

        # Pass through the fully connected layer to get logits
        out = self.fc(yhat)  # out: (batch, seq_length, vocab_size)

        return out, state

    def init_state(self, b_size=1):
        # Initializes hidden and cell states with zeros
        # Each state has shape (n_layers, batch_size, hidden_size)
        return (torch.zeros(self.n_layers, b_size, self.hidden_size),
                torch.zeros(self.n_layers, b_size, self.hidden_size))


## Evaluation 1

$random\_sample\_next$ which randomly sample the next word on $p(wn|w0, w1, . . . , wn−1)$

In [17]:

def random_sample_next(model, x, prev_state, topk=None):
    """
    Randomly samples the next word based on the probability distribution.

    Args:
        model: Trained LSTM model.
        x: Input tensor of shape (batch_size, seq_length).
        prev_state: Previous hidden and cell state of the LSTM.
        topk: Number of top candidates to consider for sampling. Defaults to all if None.

    Returns:
        sampled_ix: Index of the randomly sampled word.
        state: Updated LSTM state after processing the input.
    """
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x, prev_state)
    last_out = out[0, -1, :]  # Vocabulary values of last element of sequence

    # if topk is not None:

    # Get the top-k indexes and their values
    topk = topk if topk else last_out.shape[0]
    top_logit, top_ix = torch.topk(last_out, k=topk, dim=-1)
    # Convert logits to probabilities and sample
    p = F.softmax(top_logit.detach(), dim=-1).numpy()

    # Check if top_ix is empty
    if len(top_ix) == 0:
        raise ValueError("No valid predictions were made (top_ix is empty).")

    sampled_ix = np.random.choice(top_ix.numpy(), p=p)

    #else:
        # Use all logits for sampling
        # p = F.softmax(last_out.detach(), dim=-1).numpy()
        # sampled_ix = np.random.choice(np.arange(len(p)), p=p)

    return sampled_ix, state

$sample\_argmax$ which picks the word having the highest probability according to the distribution $p(wn|w0, w1, . . . , wn−1)$.

In [18]:
def sample_argmax(model, x, prev_state):
    """
    Samples the next word by picking the one with the highest probability (argmax strategy).

    Args:
        model: Trained LSTM model.
        x: Input tensor of shape (batch_size, seq_length).
        prev_state: Previous hidden and cell state of the LSTM.

    Returns:
        sampled_ix: Index of the word with the highest probability.
        state: Updated LSTM state after processing the input.
    """
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x, prev_state)
    last_out = out[0, -1, :]  # Vocabulary values of last element of sequence

    # Get the index with the highest probability
    sampled_ix = torch.argmax(last_out).item()

    return sampled_ix, state

$sample$ takes as minimal input: a prompt (some words), the model, and one of the two functions above defined

In [19]:
def sample(model, seed, strategy="random", topk=5, max_seqlen=18, stop_on=None):
    """
    Generates a sequence using the model.

    Args:
        model: Trained LSTM model.
        seed: Initial list of token indices to start generation.
        strategy: Sampling strategy - 'random' or 'max'.
        topk: Number of top candidates to consider for 'random' sampling.
        max_seqlen: Maximum sequence length to generate.
        stop_on: Token index to stop generation.

    Returns:
        sampled_ix_list: List of token indices for the generated sequence.
    """
    # the model expect that seed (prompt) is a list or a tuple to iter on it. But if it is a single int transform it in the correct form
    seed = seed if isinstance(seed, (list, tuple)) else [seed]
    model.eval()
    with torch.no_grad():
        sampled_ix_list = seed[:]
        x = torch.tensor([seed])

        prev_state = model.init_state(b_size=1)
        for _ in range(max_seqlen - len(seed)):
            # Repeatedly predicts the next word/token based on the input sequence
            if strategy == "random":
                sampled_ix, prev_state = random_sample_next(model, x, prev_state, topk)
            else:
                sampled_ix, prev_state = sample_argmax(model, x, prev_state)

            # The predicted token is appended to the sequence
            sampled_ix_list.append(sampled_ix)

            # The new token is used as the input for the next prediction
            x = torch.tensor([[sampled_ix]])

            # If the predicted token is word_to_int["<EOS>"] the function terminates the loop
            if stop_on is not None and sampled_ix == stop_on:
                break

    model.train()
    return sampled_ix_list

Prior to training, evaluate your model generating and reporting here 2/3 sentences in the following way

In [20]:
model = LSTMModel(word_to_int)

In [21]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps'
        if torch.backends.mps.is_available() else 'cpu')
model = model.to(DEVICE)
print("Working on", DEVICE)

Working on cuda


In [22]:
# Function to tokenize and map words to indices
def tokenize_and_map(sentence, word_to_int):
    tokens = sentence.split(" ")  # Split sentence into words
    return [word_to_int[word] for word in tokens if word in word_to_int]

def decode_sequence(generated_tokens, int_to_word):
    # Convert a list of token IDs into words using the int_to_word mapping
    return [int_to_word[token] for token in generated_tokens if token in int_to_word]

In [32]:
# Start with any prompt, e.g., “the president wants”, and generate three sentences with the sampling strategy.
seed = "the president wants"
seed = tokenize_and_map(seed, word_to_int)
print(f"Seed: {seed}")

for i in range(3):
    generated = sample(model, seed, "random", word_to_int["<EOS>"])
    generated = decode_sequence(generated, int_to_word)
    print("Generated: ", generated)

Seed: [29513, 23553, 31678]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# Start with any prompt, e.g., “the president wants”, and generate three sentences with the sampling strategy.
seed = "the president wants"
seed = tokenize_and_map(seed, word_to_int)  # Convert to token indices
print(f"Seed: {seed}")

for i in range(3):
    generated = sample(model, seed, "random", word_to_int["<EOS>"])
    generated = decode_sequence(generated, int_to_word)
    print("Generated: ", generated)

## Training

In [40]:
model.train()

costs = []
running_loss = 0
loss_hist = []

lr=1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=word_to_int["PAD"])
clip=1

print_every=2
num_epochs = 12
epoch = 0

while epoch<num_epochs:
    epoch += 1
    for x, y in dataloader:
        x, y = x.long().to(DEVICE), y.long().to(DEVICE)
        optimizer.zero_grad()
        # Initialise model's state and perform forward-prop
        prev_state = model.init_state(b_size=x.shape[0])
        prev_state = tuple(s.to(DEVICE) for s in prev_state)  # to DEVICE
        out, state = model(x, prev_state)         # out has dim: batch x seq_length x vocab_size

        # Calculate loss
        loss = criterion(out.transpose(1, 2), y)  #transpose is required to obtain batch x vocab_size x seq_length            costs.append(loss.item())
        running_loss += loss.item()

        # Calculate gradients and update parameters
        loss.backward()
        if clip:
            nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

    if print_every and (epoch%print_every)==0:
        print("Epoch: {}/{}, Loss: {:8.4f}".format(
                int(epoch), int(num_epochs),
                running_loss/float(print_every*len(dataloader))))
        loss_hist.append(running_loss/float(print_every*len(dataloader)))
        running_loss = 0

    if loss < 1.5:
            break

Epoch: 2/12, Loss:   6.3853
Epoch: 4/12, Loss:   6.1057
Epoch: 6/12, Loss:   5.8916
Epoch: 8/12, Loss:   5.7213
Epoch: 10/12, Loss:   5.5822
Epoch: 12/12, Loss:   5.4669
